# Fase 2: Limpieza de Datos

In [1]:
import pandas as pd
import numpy as np
import json

import sys
import os

# Agregar el directorio 'src' al path
sys.path.append(os.path.abspath('../src'))

# Importar la función suma del módulo auxiliares
import soporte_limpieza as sl

In [14]:
df = pd.read_csv("../datos/tablas_concatenadas.csv")
df.head(2)

,Unnamed: 0,id organizacion superior,organizacion superior,id organizacion,organizacion,id unidad gestora,unidad gestora,categoria economica,origen ingreso,tipo ingreso,valor previsto,valor registrado,valor recaudado,porcentaje recaudado,fecha recaudacion,anio recaudacion
0,0,63000.0,NaN,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",0.0,0.0,0.0,0.0,2013-12-31,2013.0
1,1,63000.0,Advocacia-Geral da União,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",0.0,0.0,0.0,0.0,2013-12-31,2013.0


In [3]:
df.isnull().sum()

Unnamed: 0                       0
id organizacion superior     30351
organizacion superior       357007
id organizacion              25111
organizacion                 34861
id unidad gestora            33566
unidad gestora               19470
categoria economica          18971
origen ingreso               38394
tipo ingreso                 31905
valor previsto               51276
valor registrado             51276
valor recaudado              51276
porcentaje recaudado         51276
fecha recaudacion            16324
anio recaudacion                 0
dtype: int64

### 1- Gestión de los nulos de id organizacion superior y organizacion superior.

Vamos a ver si podemos crear un diccionario para los id y los nombres de la organización superior.

In [4]:
len(df["id organizacion superior"].unique())

26

In [5]:
len(df["organizacion superior"].unique())

26

A priori parece que hay el mismo numero de ids que de nombres, buena señal.

In [6]:
lista_id_os = list(df["id organizacion superior"].unique())
lista_id_os

[np.float64(63000.0),
 np.float64(37000.0),
 np.float64(nan),
 np.float64(22000.0),
 np.float64(55000.0),
 np.float64(24000.0),
 np.float64(52000.0),
 np.float64(25000.0),
 np.float64(26000.0),
 np.float64(39000.0),
 np.float64(30000.0),
 np.float64(81000.0),
 np.float64(58000.0),
 np.float64(33000.0),
 np.float64(36000.0),
 np.float64(41000.0),
 np.float64(57000.0),
 np.float64(35000.0),
 np.float64(32000.0),
 np.float64(49000.0),
 np.float64(53000.0),
 np.float64(51000.0),
 np.float64(44000.0),
 np.float64(38000.0),
 np.float64(54000.0),
 np.float64(20000.0)]

In [7]:
for id in lista_id_os:
    print(f"\n{id}")
    filtro1 = df["id organizacion superior"] == float(id)
    filtro2 = df["organizacion superior"].notnull()
    print(len(df[filtro1 & filtro2]["organizacion superior"].unique()))


63000.0
1

37000.0
1

nan
0

22000.0
1

55000.0
1

24000.0
1

52000.0
1

25000.0
1

26000.0
1

39000.0
1

30000.0
1

81000.0
1

58000.0
1

33000.0
1

36000.0
1

41000.0
1

57000.0
1

35000.0
1

32000.0
1

49000.0
1

53000.0
1

51000.0
1

44000.0
1

38000.0
1

54000.0
1

20000.0
1


Como vemos que para cada id hay un unico valor creamos un diccionario:

In [8]:
dic_id_os = {}
for id in lista_id_os:
    try:
        filtro1 = df["id organizacion superior"] == float(id)
        filtro2 = df["organizacion superior"].notnull()
        dic_id_os[float(id)]=df[filtro1 & filtro2]["organizacion superior"].unique()[0]
    except:
        pass

In [9]:
dic_id_os

{63000.0: 'Advocacia-Geral da União',
 37000.0: 'Controladoria-Geral da União',
 22000.0: 'Ministério da Agricultura, Pecuária e Abastec',
 55000.0: 'Ministério da Cidadania',
 24000.0: 'Ministério da Ciência, Tecnologia, Inovações ',
 52000.0: 'Ministério da Defesa',
 25000.0: 'Ministério da Economia',
 26000.0: 'Ministério da Educação',
 39000.0: 'Ministério da Infraestrutura',
 30000.0: 'Ministério da Justiça e Segurança Pública',
 81000.0: 'Ministério da Mulher, Família e Direitos Huma',
 58000.0: 'Ministério da Pesca e Aquicultura',
 33000.0: 'Ministério da Previdência Social',
 36000.0: 'Ministério da Saúde',
 41000.0: 'Ministério das Comunicações',
 57000.0: 'Ministério das Mulheres, Igualdade Racial, da',
 35000.0: 'Ministério das Relações Exteriores',
 32000.0: 'Ministério de Minas e Energia',
 49000.0: 'Ministério do Desenvolvimento Agrário',
 53000.0: 'Ministério do Desenvolvimento Regional',
 51000.0: 'Ministério do Esporte',
 44000.0: 'Ministério do Meio Ambiente',
 38000.

Ya con nuestro diccionario vamos a rellenar los nombres e id de las organizaciones superiores.

Hemos comprobado que todos los campos de organización superior concuerdan con su id a excepción de cuando son NaN, corregimos los NaN. además quedarán sin corregir aquellas filas donde id y organización superioir son ambos NaN.

In [17]:
# filas que van a quedar sin corregir.
filtro1 = df["id organizacion superior"].isnull()
filtro2 = df["organizacion superior"].isnull()
df[filtro1 & filtro2].shape

(10543, 16)

In [18]:
df["organizacion superior"].fillna(df["id organizacion superior"].map(dic_id_os), inplace=True)

C:\Users\Elena\AppData\Local\Temp\ipykernel_7136\3684483159.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["organizacion superior"].fillna(df["id organizacion superior"].map(dic_id_os), inplace=True)


In [24]:
os_null = df[df["organizacion superior"].isnull()].shape[0]
os_sin_corregir = os_null/df.shape[0]*100
print(f"Han quedado a null {os_null}, es decir, un {np.round(os_sin_corregir,2)}% del total de las organizaciones superiores.")

Han quedado a null 10543, es decir, un 1.04% del total de las organizaciones superiores.


In [25]:
df[df["id organizacion superior"].isnull()].shape[0]

30351

Como podemos ver ids hay 30351 a null los cuales podemos corregir al rededor de unos 20000 a partir del nombre de la organizacion superior.

In [26]:
#Invertimos el diccionario
dic_organizaciones_id = {valor: clave for clave, valor in dic_id_os.items()}
dic_organizaciones_id


{'Advocacia-Geral da União': 63000.0,
 'Controladoria-Geral da União': 37000.0,
 'Ministério da Agricultura, Pecuária e Abastec': 22000.0,
 'Ministério da Cidadania': 55000.0,
 'Ministério da Ciência, Tecnologia, Inovações ': 24000.0,
 'Ministério da Defesa': 52000.0,
 'Ministério da Economia': 25000.0,
 'Ministério da Educação': 26000.0,
 'Ministério da Infraestrutura': 39000.0,
 'Ministério da Justiça e Segurança Pública': 30000.0,
 'Ministério da Mulher, Família e Direitos Huma': 81000.0,
 'Ministério da Pesca e Aquicultura': 58000.0,
 'Ministério da Previdência Social': 33000.0,
 'Ministério da Saúde': 36000.0,
 'Ministério das Comunicações': 41000.0,
 'Ministério das Mulheres, Igualdade Racial, da': 57000.0,
 'Ministério das Relações Exteriores': 35000.0,
 'Ministério de Minas e Energia': 32000.0,
 'Ministério do Desenvolvimento Agrário': 49000.0,
 'Ministério do Desenvolvimento Regional': 53000.0,
 'Ministério do Esporte': 51000.0,
 'Ministério do Meio Ambiente': 44000.0,
 'Minis

In [27]:
df["id organizacion superior"].fillna(df["organizacion superior"].map(dic_organizaciones_id), inplace=True)

C:\Users\Elena\AppData\Local\Temp\ipykernel_7136\1809695679.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["id organizacion superior"].fillna(df["organizacion superior"].map(dic_organizaciones_id), inplace=True)


In [28]:
df[df["id organizacion superior"].isnull()].shape[0]

10543

Comprobamos cuantos null hay ahora y efectivamente se han reducido hasta 10543, los mismo que para los nombres de las organizaciones superiores ya que son todas las filas que tienen ambos campos a null. Aunque al represenatr un 1% y no haber forma de arreglarlo puede ser despreciable.

### 2- Gestión de los nulos de id organizacion y organizacion.

Vamos a seguir la misma estrategia que en el paso anterior.

# Conclusiones

1- Hemos corregido los nombres de las organizaciones superiores a partir de los id y han quedado solo 10543 sin corregir lo que representa un 1.04% del total.